<a href="https://colab.research.google.com/github/AA-RGCL/Forged-GAN-BERT/blob/main/notebooks/ChatGPT_AA_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate Dataset

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 13.1 MB/s eta 0:00:00


In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 6.1 MB/s eta 0:00:00


In [ ]:
!pip install gutenberg_cleaner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split

import nltk
nltk.download('punkt')

import openai
import os
import re

In [ ]:
!huggingface-cli login

In [ ]:
# Set up your API key
openai.api_key = <<OPEN_AI_API_KEY>>

In [ ]:
def generate_similar_novel(book_name, author):
  # prompt = (f"Write a complete novel with similar to {book_name} by {author}. The writing style should be the same in novel. The novel should be around 12,000 words.")
  prompt = (f"Write a complete novel similar to {book_name} by {author}. The writing style should be the same as the {book_name} by {author}. The novel should be than 12,000 words.")
  # maybe remove the word counts?
  # write the first chapter
  # write the ending
  # prompt 2 - start writing a novel (remove word count)
  # prompt 3 - complete, at least, at most

  model_engine = "GPT-3.5-turbo" # Replace with the model you want to use
  prompt_length = len(nltk.word_tokenize(prompt))
  max_length = 4000

  # TODO: use the longest prompt length
  response_length = max_length - prompt_length

  print(f'Generating novel similar to {book_name} by {author}')

  response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [
      {'role': 'user', 'content': prompt}
    ],
    temperature = 0  ,
    max_tokens=response_length
  )

  return response.choices[0].message.content

# Get Huggingface Data

In [ ]:
dataset = load_dataset("Authorship/experiment_1", data_dir='authors_20/trial_1')
data = pd.concat([pd.DataFrame(dataset['train']), pd.DataFrame(dataset['test']), pd.DataFrame(dataset['validation'])])

author_names = ['Arthur Conan Doyle', 'Henry Rider Haggard', 'Jack London', 'Mark Twain', 'Wilkie Collins']
data = data[data['Name'].isin(author_names)]

data.to_csv('/content/original_texts.csv')

trainval, test = train_test_split(data,
                                test_size=0.2,
                                stratify=data['AuthorID'])

train, val = train_test_split(trainval, test_size=0.1, stratify=trainval['AuthorID'])

  0%|          | 0/3 [00:00<?, ?it/s]

# generate chat gpt texts

In [ ]:
train['ChatGPTText'] = train.apply(lambda x: generate_similar_novel(x['BookTitle'], x['Name']), axis=1)

Generating novel similar to The Disappearance of Lady Frances Carfax by Arthur Conan Doyle
Generating novel similar to She and Allan by Henry Rider Haggard
Generating novel similar to Sketches New and Old by Mark Twain
Generating novel similar to A Horse's Tale by Mark Twain
Generating novel similar to Smoke Bellew by Jack London
Generating novel similar to The Adventures of Sherlock Holmes by Arthur Conan Doyle
Generating novel similar to The Adventure of the Red Circle by Arthur Conan Doyle
Generating novel similar to The People of the Mist by Henry Rider Haggard
Generating novel similar to Adventure by Jack London
Generating novel similar to Tom Sawyer, Detective by Mark Twain
Generating novel similar to The Sea-Wolf by Jack London
Generating novel similar to A Study in Scarlet by Arthur Conan Doyle
Generating novel similar to Tom Sawyer Abroad by Mark Twain
Generating novel similar to The Croxley Master: A Great Tale Of The Prize Ring by Arthur Conan Doyle
Generating novel similar 

In [ ]:
len(train)

72

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72 entries, 157 to 209
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   BookID       72 non-null     int64 
 1   AuthorID     72 non-null     int64 
 2   Name         72 non-null     object
 3   BookTitle    72 non-null     object
 4   Genre        72 non-null     object
 5   BookText     72 non-null     object
 6   Source       72 non-null     object
 7   ChatGPTText  72 non-null     object
dtypes: int64(2), object(6)
memory usage: 5.1+ KB


In [ ]:
train.head()

,BookID,AuthorID,Name,BookTitle,Genre,BookText,Source,ChatGPTText
157,2348,69,Arthur Conan Doyle,The Disappearance of Lady Frances Carfax,Novel,The Project Gutenberg EBook of The Disappearan...,gutenberg,Chapter 1\n\nIt was a cold and dreary day in L...
117,5745,365,Henry Rider Haggard,She and Allan,Novel,"﻿The Project Gutenberg eBook of She and Allan,...",gutenberg,Chapter 1\n\nThe sun was setting over the vast...
47,3189,53,Mark Twain,Sketches New and Old,Novel,﻿The Project Gutenberg EBook of Sketches New a...,gutenberg,Title: Tales from the Mississippi\n\nChapter 1...
11,1086,53,Mark Twain,A Horse's Tale,Novel,﻿The Project Gutenberg eBook of A Horse’s Tale...,gutenberg,Chapter 1\n\nI was born in a small village in ...
86,5737,120,Jack London,Smoke Bellew,Novel,"﻿The Project Gutenberg EBook of Smoke Bellew, ...",gutenberg,Chapter 1\n\nThe sun was setting over the vast...


In [ ]:
len(test)

20

In [ ]:
test['ChatGPTText'] = test.apply(lambda x: generate_similar_novel(x['BookTitle'], x['Name']), axis=1)

Generating novel similar to The Ghost Kings by Henry Rider Haggard
Generating novel similar to The Prince and the Pauper by Mark Twain
Generating novel similar to Antonina; Or, The Fall of Rome by Wilkie Collins
Generating novel similar to The Adventure of the Dying Detective by Arthur Conan Doyle
Generating novel similar to The Woman in White by Wilkie Collins
Generating novel similar to Extract from Captain Stormfield's Visit to Heaven by Mark Twain
Generating novel similar to The Call of the Wild by Jack London
Generating novel similar to The Firm of Girdlestone by Arthur Conan Doyle
Generating novel similar to The Son of the Wolf by Jack London
Generating novel similar to The Scarlet Plague by Jack London
Generating novel similar to His Last Bow: An Epilogue of Sherlock Holmes by Arthur Conan Doyle
Generating novel similar to The Law and the Lady by Wilkie Collins
Generating novel similar to The Moonstone by Wilkie Collins
Generating novel similar to A Yellow God: An Idol of Africa

In [ ]:
test.head()

,BookID,AuthorID,Name,BookTitle,Genre,BookText,Source,ChatGPTText
237,8184,365,Henry Rider Haggard,The Ghost Kings,Novel,﻿The Project Gutenberg eBook of The Ghost King...,gutenberg,Chapter 1\n\nThe sun was setting over the vast...
71,1837,53,Mark Twain,The Prince and the Pauper,Novel,﻿\r\nThe Project Gutenberg EBook of The Prince...,gutenberg,Title: The Heir and the Orphan\n\nChapter 1\n\...
53,3606,98,Wilkie Collins,"Antonina; Or, The Fall of Rome",Novel,"The Project Gutenberg EBook of Antonina, by Wi...",gutenberg,Title: The Tragedy of Alexandria\n\nChapter 1\...
186,2347,69,Arthur Conan Doyle,The Adventure of the Dying Detective,Novel,The Project Gutenberg EBook of The Adventure o...,gutenberg,Chapter 1\n\nIt was a cold and dreary day in L...
8,583,98,Wilkie Collins,The Woman in White,Novel,The Project Gutenberg EBook of The Woman in Wh...,gutenberg,Chapter 1\n\nIt was a dark and stormy night wh...


In [ ]:
len(val)

8

In [ ]:
val['ChatGPTText'] = val.apply(lambda x: generate_similar_novel(x['BookTitle'], x['Name']), axis=1)

Generating novel similar to The American Claimant by Mark Twain
Generating novel similar to Before Adam by Jack London
Generating novel similar to The Adventure of the Cardboard Box by Arthur Conan Doyle
Generating novel similar to No Name by Wilkie Collins
Generating novel similar to Hide and Seek by Wilkie Collins
Generating novel similar to The Doings of Raffles Haw by Arthur Conan Doyle
Generating novel similar to She by Henry Rider Haggard
Generating novel similar to Morning Star by Henry Rider Haggard


In [ ]:
val.head()

,BookID,AuthorID,Name,BookTitle,Genre,BookText,Source,ChatGPTText
17,3179,53,Mark Twain,The American Claimant,Novel,﻿The Project Gutenberg eBook of The American C...,gutenberg,Chapter 1\n\nThe sun was setting over the smal...
253,310,120,Jack London,Before Adam,Novel,"﻿The Project Gutenberg eBook of Before Adam, b...",gutenberg,Chapter 1\n\nI was born in a world that was no...
149,2344,69,Arthur Conan Doyle,The Adventure of the Cardboard Box,Novel,Project Gutenberg's The Adventure of the Cardb...,gutenberg,Chapter 1\n\nIt was a hot summer day in London...
1,1438,98,Wilkie Collins,No Name,Novel,"﻿\r\nThe Project Gutenberg EBook of No Name, b...",gutenberg,Chapter 1\n\nThe sun was setting over the smal...
72,7893,98,Wilkie Collins,Hide and Seek,Novel,"﻿The Project Gutenberg EBook of Hide and Seek,...",gutenberg,Chapter 1\n\nIt was a dark and stormy night wh...


In [ ]:
huggingface_repo = '/content/ChatGPT'

In [ ]:
# train_test splits
train.to_csv(f'{huggingface_repo}/train_gpt.csv', index=False)


In [ ]:
test.to_csv(f'{huggingface_repo}/test_gpt.csv', index=False)

In [ ]:

val.to_csv(f'{huggingface_repo}/val_gpt.csv', index=False)

# Preprocess Dataset

In [ ]:
# set BookText to OldBookText and cleaned text as BookText

In [ ]:
from gutenberg_cleaner import super_cleaner
import nltk
from nltk.corpus import stopwords
import re
import string
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def preprocess_text(text):
  text_data = super_cleaner(text, max_token=len(text))
  text_data = text_data.replace('[deleted]', '')

  # to lowercase
  text_data = text_data.lower()

  # replace hyphen
  text_data = text_data.replace("-"," ")

  # remove stop words (only when it includes punctuation)
  for w in stopwords.words("english"):
      if re.match("(^|\w+)[%s](\w+|$)" % re.escape(string.punctuation), w):
          text_data = re.sub(r"(^|\s+)%s(\s+|$)" % re.escape(w)," ",text_data)
  text_data = text_data.strip()

  # remove punctuation
  text_data = re.sub(r"[%s]" % re.escape(string.punctuation),"",text_data)
  text_data = text_data.strip()

  # remove stop words (only when it doesn't include punctuation)
  for w in stopwords.words("english"):
      if not re.match("(^|\w+)[%s](\w+|$)" % re.escape(string.punctuation), w):
          text_data = re.sub(r"(^|\s+)%s(\s+|$)" % re.escape(w)," ",text_data)
  text_data = text_data.strip()

  text_data = text_data.replace('\n', ' ').replace('\r', '')

  return text_data

In [ ]:
def preprocess_df(df):
  df.rename(columns={"BookText": "OldBookText"}, inplace=True)
  df['BookText'] = df.OldBookText.apply(lambda x: preprocess_text(x))

In [ ]:
def preprocess_chatgpt_df(df):
  df.rename(columns={"ChatGPTText": "OldChatGPTText"}, inplace=True)
  df['ChatGPTText'] = df.OldChatGPTText.apply(lambda x: preprocess_text(x))

In [ ]:
test = pd.read_csv('/content/ChatGPT/test.csv')
test_gpt = pd.read_csv('/content/ChatGPT/test_gpt.csv')
train = pd.read_csv('/content/ChatGPT/train.csv')
train_gpt = pd.read_csv('/content/ChatGPT/train_gpt.csv')
val = pd.read_csv('/content/ChatGPT/val.csv')
val_gpt = pd.read_csv('/content/ChatGPT/val_gpt.csv')

In [ ]:
preprocess_df(train)

In [ ]:
train_gpt['AuthorID'].unique()

array([ 69, 365,  53, 120,  98])

In [ ]:
preprocess_df(train_gpt)

In [ ]:
preprocess_chatgpt_df(train_gpt)

In [ ]:
train_gpt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   BookID          72 non-null     int64 
 1   AuthorID        72 non-null     int64 
 2   Name            72 non-null     object
 3   BookTitle       72 non-null     object
 4   Genre           72 non-null     object
 5   OldBookText     72 non-null     object
 6   Source          72 non-null     object
 7   OldChatGPTText  72 non-null     object
 8   BookText        72 non-null     object
 9   ChatGPTText     72 non-null     object
dtypes: int64(2), object(8)
memory usage: 5.8+ KB


In [ ]:
preprocess_df(test)
preprocess_df(test_gpt)

In [ ]:
preprocess_chatgpt_df(test_gpt)

In [ ]:
preprocess_df(val)
preprocess_df(val_gpt)

In [ ]:
preprocess_chatgpt_df(val_gpt)

In [ ]:
val_gpt.head()

,BookID,AuthorID,Name,BookTitle,Genre,OldBookText,Source,OldChatGPTText,BookText,ChatGPTText
0,3179,53,Mark Twain,The American Claimant,Novel,﻿The Project Gutenberg eBook of The American C...,gutenberg,Chapter 1\n\nThe sun was setting over the smal...,chapter earl rossmore vs american claimant—vis...,sun setting small town millville casting warm ...
1,310,120,Jack London,Before Adam,Novel,"﻿The Project Gutenberg eBook of Before Adam, b...",gutenberg,Chapter 1\n\nI was born in a world that was no...,“these ancestors history history remember sure...,born world like world today world wildness dan...
2,2344,69,Arthur Conan Doyle,The Adventure of the Cardboard Box,Novel,Project Gutenberg's The Adventure of the Cardb...,gutenberg,Chapter 1\n\nIt was a hot summer day in London...,adventure cardboard box choosing typical cases...,hot summer day london sherlock holmes sitting ...
3,1438,98,Wilkie Collins,No Name,Novel,"﻿\r\nThe Project Gutenberg EBook of No Name, b...",gutenberg,Chapter 1\n\nThe sun was setting over the smal...,main purpose story appeal reader’s interest su...,sun setting small village oakwood casting warm...
4,7893,98,Wilkie Collins,Hide and Seek,Novel,"﻿The Project Gutenberg EBook of Hide and Seek,...",gutenberg,Chapter 1\n\nIt was a dark and stormy night wh...,novel ranks third order succession works ficti...,dark stormy night young beautiful emily arrive...


In [ ]:
huggingface_repo = '/content/ChatGPT'

In [ ]:
# train_test splits
train_gpt.to_csv(f'{huggingface_repo}/train_gpt.csv', index=False)
test_gpt.to_csv(f'{huggingface_repo}/test_gpt.csv', index=False)
val_gpt.to_csv(f'{huggingface_repo}/val_gpt.csv', index=False)

In [ ]:
train.to_csv(f'{huggingface_repo}/train.csv', index=False)
test.to_csv(f'{huggingface_repo}/test.csv', index=False)
val.to_csv(f'{huggingface_repo}/val.csv', index=False)

# RQ1: ChatGPT vs Author - Binary

In [ ]:
train,test,val = train_gpt,test_gpt,val_gpt

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   BookID          72 non-null     int64 
 1   AuthorID        72 non-null     int64 
 2   Name            72 non-null     object
 3   BookTitle       72 non-null     object
 4   Genre           72 non-null     object
 5   OldBookText     72 non-null     object
 6   Source          72 non-null     object
 7   OldChatGPTText  72 non-null     object
 8   BookText        72 non-null     object
 9   ChatGPTText     72 non-null     object
dtypes: int64(2), object(8)
memory usage: 5.8+ KB


In [ ]:
def create_binary_datasets(author_name, dataset, path):
  df_ACD_train = dataset[dataset.Name ==author_name]

  df_gpt = df_ACD_train[['BookID', 'AuthorID', 'Name','BookTitle', 'Genre', 'ChatGPTText', 'Source']]
  df_gpt.AuthorID = 9999
  df_gpt.Name = 'Chat GPT'
  df_gpt['BookTitle'] = df_gpt.apply(lambda x: 'ChatGPT_{0}'.format(x['BookTitle']), axis=1)
  df_gpt.Source = 'Chat GPT'
  df_gpt = df_gpt.rename(columns={'ChatGPTText': 'BookText'})

  df_ACD_train.drop(columns=['ChatGPTText', 'OldChatGPTText', 'OldBookText'], axis=1, inplace=True)
  result = pd.concat([df_ACD_train, df_gpt])

  result.to_csv(f'{path}', index=False)

In [ ]:
def create_binary_dataset_train_test_splits(author_name,author_initials, train, test, val):
  create_binary_datasets(author_name, train, f'/content/ChatGPT/binary/{author_initials}/train.csv')
  create_binary_datasets(author_name, test, f'/content/ChatGPT/binary/{author_initials}/test.csv')
  create_binary_datasets(author_name, val, f'/content/ChatGPT/binary/{author_initials}/val.csv')

In [ ]:
# ACD
create_binary_dataset_train_test_splits('Arthur Conan Doyle','ACD', train,test,val)

# HRD
create_binary_dataset_train_test_splits('Henry Rider Haggard','HRD', train,test,val)

# JL
create_binary_dataset_train_test_splits('Jack London','JL', train,test,val)

# MT
create_binary_dataset_train_test_splits('Mark Twain','MT', train,test,val)

# WK
create_binary_dataset_train_test_splits('Wilkie Collins','WK', train,test,val)

<ipython-input-114-64e6268072fc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gpt.AuthorID = 9999
<ipython-input-114-64e6268072fc>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gpt.Name = 'Chat GPT'
<ipython-input-114-64e6268072fc>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

# RQ2: ChatGPT with Others - Multi

In [ ]:
def create_multi_datasets(dataset, path):
  df_gpt = dataset[['BookID', 'AuthorID', 'Name','BookTitle', 'Genre', 'ChatGPTText', 'Source']]
  df_gpt.AuthorID = 9999
  df_gpt.Name = 'Chat GPT'
  df_gpt['BookTitle'] = df_gpt.apply(lambda x: 'ChatGPT_{0}'.format(x['BookTitle']), axis=1)
  df_gpt.Source = 'Chat GPT'
  df_gpt = df_gpt.rename(columns={'ChatGPTText': 'BookText'})

  dataset.drop(columns=['ChatGPTText', 'OldChatGPTText', 'OldBookText'], axis=1, inplace=True)
  result = pd.concat([dataset, df_gpt])

  result.to_csv(f'{path}', index=False)

In [ ]:
create_multi_datasets(train, '/content/ChatGPT/multi/train.csv')
create_multi_datasets(test, '/content/ChatGPT/multi/test.csv')
create_multi_datasets(val, '/content/ChatGPT/multi/val.csv')

<ipython-input-122-782e14d1c459>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gpt.AuthorID = 9999
<ipython-input-122-782e14d1c459>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gpt.Name = 'Chat GPT'
<ipython-input-122-782e14d1c459>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni